In [0]:
import pyspark.sql.functions as F
from datetime import datetime, timedelta
from pyspark.sql import SparkSession

In [0]:
df_origem = spark.table("estudo.default.torigem_tratada").withColumn("media_tickets_por_compra", F.round(F.col("total_tickets_compras") / F.col("total_compras"), 2)) \
    .withColumn("porcentagem_viagens_com_volta", F.round(F.col("total_compras_com_volta") / F.col("total_compras") * 100, 2)) \
        .select(
            "fk_contact",
            "valor_medio_compra",
            "total_tickets_compras",
            "media_tickets_por_compra",
            "total_compras_com_volta",
            "porcentagem_viagens_com_volta",
            "local_partida_ida_local_destino_ida_frequente",
            "local_partida_volta_local_destino_volta_frequente",
            "dia_da_semana_preferido_de_compra",
            "epoca_do_mes_preferido_da_compra",
            "periodo_do_dia_preferido_para_compra",
            "epoca_do_ano_preferido_da_compra"
        ).distinct()

In [0]:
df_cluster = spark.table("estudo.default.tuser_clustering")

In [0]:
df_join = df_origem \
    .join(df_cluster, on=["fk_contact"], how="left")

In [0]:
df_final = df_join.select(
        "fk_contact",
        F.col("cluster").alias("cd_cluster"),
        F.col("segmento_do_cliente").alias("ds_cluster"),
        "recencia_dias",
        F.round("valor_total_gasto", 2).alias("valor_total_gasto"),
        F.round("valor_medio_compra", 2).alias("valor_medio_compra"),
        F.col("frequencia_de_compras").alias("quantidade_compras"),
        "total_tickets_compras",
        "media_tickets_por_compra",
        "total_compras_com_volta",
        "porcentagem_viagens_com_volta",
        F.col("local_partida_ida_local_destino_ida_frequente").alias("rota_ida_frequente"),
        F.col("local_partida_volta_local_destino_volta_frequente").alias("rota_volta_frequente"),
        F.col("dia_da_semana_preferido_de_compra").alias("dia_semana_preferido_compra"),
        F.col("epoca_do_mes_preferido_da_compra").alias("dia_mes_preferido_compra"),
        F.col("periodo_do_dia_preferido_para_compra").alias("periodo_dia_preferido_compra"),
        F.col("epoca_do_ano_preferido_da_compra").alias("mes_preferido_compra")
    )

In [0]:
df_final.count()

In [0]:
df_final.write.mode("overwrite").saveAsTable("estudo.default.tperfil_cliente")

In [0]:
df_final.filter(F.col('total_compras_com_volta') > 1).display()

In [0]:
df_final.filter(F.col('fk_contact') == 'cbae228abf4b9f465f42d8d6f97a83f2b76c077aea36c3b7303915796cf3dac3').display()